#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [3]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_2_6')

# Define the destination directory
# This should be the import directory of your graph database
#Windows
#destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/import'

#Linux
destination_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [4]:
# Define the working directory
#Windows
#working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/bin'

#Linux
working_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361'

# Construct the command Linux
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv',
    '--overwrite-destination',  'neo4j'
]

#Windows
#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'

# Execute the command


#result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)
result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361/data/databases/neo4j:\nNodes:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361/import/nodes.csv\n\nRelationships:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 15.47GiB\n  Free machine memory: 1.381GiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 8\n  Configured max memory: 653.4MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'Weight' normalized from 'float' --> 'double' in /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-b10bd033-c28d-4fe4-aefd-32b992c43361/import/relationships.csv\n\nImport s

Run and connect to the Neo4j Database

In [5]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [6]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [16]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SurID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [17]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SurID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [18]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SurID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [19]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SurID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [20]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SurID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [21]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SurID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [22]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SurID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [23]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SurID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [24]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SurID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [25]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [26]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

In [27]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['name'], how='left')

In [40]:
df = merged_df.rename(columns={'name' : 'SurID'})
df

,SurID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210120-2,1.066556,50.933572,0.141482,0.244373,1,18
1,210505-1,1.061224,50.639134,0.140664,0.243825,1,18
2,210303-2,1.057692,50.437632,0.140105,0.243457,1,18
3,210324-1,1.055017,50.304457,0.139735,0.243198,1,18
4,210504-1,1.050611,50.071016,0.139086,0.242759,1,18
...,...,...,...,...,...,...,...
58,210429-1,0.826333,37.573935,0.104372,0.219628,0,43
59,210407-1,0.825272,37.471294,0.104087,0.219450,0,43
60,210311-1,0.822136,37.324738,0.103680,0.219162,1,18
61,210518-1,0.813073,36.805702,0.102238,0.218214,1,18


In [41]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SurID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SurID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1,0.148074,0.062992,-0.086053,-0.013960,-0.094132,0.093681,-0.076875,0.099170,-0.104518,...,0.362389,-0.045680,-0.114490,0.136893,-0.061356,-0.126552,-0.119260,-0.119269,-0.084268,0.051636
1,201210-2,0.148044,0.063032,-0.086974,-0.013786,-0.092909,0.094044,-0.076408,0.099270,-0.103982,...,0.362324,-0.046032,-0.112472,0.136427,-0.061472,-0.126235,-0.118634,-0.120265,-0.085383,0.052064
2,210114-1,0.147864,0.062416,-0.086824,-0.013481,-0.093673,0.094785,-0.076168,0.099154,-0.103975,...,0.362557,-0.046539,-0.112776,0.136170,-0.061643,-0.126523,-0.119153,-0.119744,-0.084853,0.051985
3,210114-2,0.147349,0.062351,-0.086448,-0.013313,-0.093593,0.094751,-0.076623,0.099181,-0.103776,...,0.362583,-0.046065,-0.113099,0.136091,-0.061563,-0.127183,-0.118543,-0.119781,-0.085047,0.052079
4,210120-1,0.148864,0.065607,-0.086003,-0.013842,-0.091871,0.094693,-0.076708,0.098820,-0.106134,...,0.361995,-0.046259,-0.110904,0.137079,-0.061162,-0.125793,-0.119311,-0.122582,-0.086542,0.052334


Delete the projection

In [23]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [24]:
close_driver()

In [42]:
status_df = pd.read_csv('../../data/graph_2_6/status.csv')

In [43]:
status_df

,Unnamed: 0,SurID,Status
0,0,201210-1,Normal
1,1,201210-2,Normal
2,2,210114-1,Normal
3,3,210114-2,Normal
4,4,210120-1,Hyperglycemia
...,...,...,...
58,58,210519-3,Hyperglycemia
59,59,210524-1,Hypoglycemia
60,60,210526-1,Hyperglycemia
61,61,210526-2,Hyperglycemia


In [44]:
df

,SurID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210120-2,1.066556,50.933572,0.141482,0.244373,1,18
1,210505-1,1.061224,50.639134,0.140664,0.243825,1,18
2,210303-2,1.057692,50.437632,0.140105,0.243457,1,18
3,210324-1,1.055017,50.304457,0.139735,0.243198,1,18
4,210504-1,1.050611,50.071016,0.139086,0.242759,1,18
...,...,...,...,...,...,...,...
58,210429-1,0.826333,37.573935,0.104372,0.219628,0,43
59,210407-1,0.825272,37.471294,0.104087,0.219450,0,43
60,210311-1,0.822136,37.324738,0.103680,0.219162,1,18
61,210518-1,0.813073,36.805702,0.102238,0.218214,1,18


In [45]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [47]:
graph_df = pd.merge(df, status_df, left_on="SurID", right_on="SurID", how="inner")
#graph_df.drop(columns=["SurID:ID"], inplace=True)

In [49]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="SurID", right_on="SurID", how="inner")
#fast_graph_df.drop(columns=["SurID:ID"], inplace=True)

In [50]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [51]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [52]:
fast_graph_df

,SurID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status
0,201210-1,0.148074,0.062992,-0.086053,-0.013960,-0.094132,0.093681,-0.076875,0.099170,-0.104518,...,-0.045680,-0.114490,0.136893,-0.061356,-0.126552,-0.119260,-0.119269,-0.084268,0.051636,Normal
1,201210-2,0.148044,0.063032,-0.086974,-0.013786,-0.092909,0.094044,-0.076408,0.099270,-0.103982,...,-0.046032,-0.112472,0.136427,-0.061472,-0.126235,-0.118634,-0.120265,-0.085383,0.052064,Normal
2,210114-1,0.147864,0.062416,-0.086824,-0.013481,-0.093673,0.094785,-0.076168,0.099154,-0.103975,...,-0.046539,-0.112776,0.136170,-0.061643,-0.126523,-0.119153,-0.119744,-0.084853,0.051985,Normal
3,210114-2,0.147349,0.062351,-0.086448,-0.013313,-0.093593,0.094751,-0.076623,0.099181,-0.103776,...,-0.046065,-0.113099,0.136091,-0.061563,-0.127183,-0.118543,-0.119781,-0.085047,0.052079,Normal
4,210120-1,0.148864,0.065607,-0.086003,-0.013842,-0.091871,0.094693,-0.076708,0.098820,-0.106134,...,-0.046259,-0.110904,0.137079,-0.061162,-0.125793,-0.119311,-0.122582,-0.086542,0.052334,Hyperglycemia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,210519-3,0.147535,0.062793,-0.086827,-0.013634,-0.093383,0.093621,-0.076320,0.099288,-0.104212,...,-0.046118,-0.112944,0.136373,-0.061016,-0.126432,-0.118645,-0.120671,-0.084766,0.052149,Hyperglycemia
59,210524-1,0.148089,0.063263,-0.086243,-0.013925,-0.093347,0.094211,-0.076082,0.099329,-0.104206,...,-0.045913,-0.113085,0.135863,-0.061051,-0.126128,-0.118741,-0.119799,-0.085271,0.051936,Hypoglycemia
60,210526-1,0.149026,0.064424,-0.085681,-0.014899,-0.091819,0.093848,-0.077155,0.099565,-0.105782,...,-0.046011,-0.111519,0.137123,-0.061071,-0.125619,-0.119364,-0.121773,-0.087036,0.052698,Hyperglycemia
61,210526-2,0.149709,0.064195,-0.085909,-0.015113,-0.093363,0.093056,-0.077332,0.099563,-0.105623,...,-0.045630,-0.113384,0.136593,-0.060970,-0.126464,-0.119272,-0.120642,-0.085524,0.051788,Hyperglycemia


In [53]:
graph_df

,SurID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status
0,210120-2,1.066556,50.933572,0.141482,0.244373,1,18,Hyperglycemia
1,210505-1,1.061224,50.639134,0.140664,0.243825,1,18,Hypoglycemia
2,210303-2,1.057692,50.437632,0.140105,0.243457,1,18,Normal
3,210324-1,1.055017,50.304457,0.139735,0.243198,1,18,Normal
4,210504-1,1.050611,50.071016,0.139086,0.242759,1,18,Hypoglycemia
...,...,...,...,...,...,...,...,...
58,210429-1,0.826333,37.573935,0.104372,0.219628,0,43,Normal
59,210407-1,0.825272,37.471294,0.104087,0.219450,0,43,Hypoglycemia
60,210311-1,0.822136,37.324738,0.103680,0.219162,1,18,Normal
61,210518-1,0.813073,36.805702,0.102238,0.218214,1,18,Hyperglycemia


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = graph_df['SurID']
    X = graph_df.drop(['Status', 'SurID', 'SpecID'], axis=1)
    y = graph_df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = []
    for train_index, test_index in cv.split(X, y, groups):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Extra Trees classifier
        et.fit(X_train, y_train)
        predictions = et.predict(X_test)
        
        # Evaluate the model
        score = accuracy_score(y_test, predictions)
        scores.append(score)
    
    # Displaying the results
    print(f'{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

In [55]:
evaluate_extra_trees(graph_df)

KeyError: "['SpecID'] not found in axis"

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = graph_df.drop(['Status', 'SurID'], axis=1)
y = graph_df['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.2357 +/- 0.2191
ExtraTreesClassifier Cross-Validation Accuracy: 0.2381 +/- 0.2564


In [37]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [57]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [58]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [59]:
calculate_metrics(y_test, rf_predictions)

NameError: name 'y_test' is not defined

In [41]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.3694581280788177


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.35      0.31      0.33       192
 Hypoglycemia       0.39      0.38      0.38       218
       Normal       0.37      0.42      0.39       199

     accuracy                           0.37       609
    macro avg       0.37      0.37      0.37       609
 weighted avg       0.37      0.37      0.37       609


Confusion Matrix:
[[60 65 67]
 [62 82 74]
 [51 65 83]]


In [42]:
get_feature_importances(rf_model, X)

,Feature,Importance
0,PageRank,0.242765
1,DegreeCentrality,0.230733
3,ArticleRank,0.230634
2,EigenvectorCentrality,0.230235
4,Leiden,0.035406
5,Louvain,0.030227


In [43]:
get_feature_importances(et_model, X)

,Feature,Importance
0,PageRank,0.240788
3,ArticleRank,0.235372
1,DegreeCentrality,0.234854
2,EigenvectorCentrality,0.230949
4,Leiden,0.031944
5,Louvain,0.026093


## FastRp

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['Status', 'SurID'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.38461538461538464
Extra Trees Accuracy: 0.23076923076923078


In [ ]:
evaluate_extra_trees(fast_graph_df)

Fast rp mixed with traditional graph metrics

In [63]:
joined_graph = pd.merge(graph_df, fast_graph_df, on=['Status', 'SurID'], how='inner')

In [67]:
joined_graph

,SurID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,embedding_0,embedding_1,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210120-2,1.066556,50.933572,0.141482,0.244373,1,18,Hyperglycemia,0.147939,0.062011,...,0.362641,-0.046410,-0.113797,0.135514,-0.061132,-0.126738,-0.118488,-0.119333,-0.084253,0.051966
1,210505-1,1.061224,50.639134,0.140664,0.243825,1,18,Hypoglycemia,0.147809,0.062583,...,0.362552,-0.046113,-0.112984,0.136304,-0.061699,-0.126571,-0.118129,-0.119452,-0.085056,0.052088
2,210303-2,1.057692,50.437632,0.140105,0.243457,1,18,Normal,0.147792,0.062412,...,0.362513,-0.045952,-0.113445,0.136621,-0.060664,-0.126634,-0.118452,-0.119566,-0.084867,0.051928
3,210324-1,1.055017,50.304457,0.139735,0.243198,1,18,Normal,0.147924,0.061721,...,0.362956,-0.045996,-0.113928,0.136583,-0.061008,-0.126527,-0.118501,-0.120097,-0.084825,0.052026
4,210504-1,1.050611,50.071016,0.139086,0.242759,1,18,Hypoglycemia,0.147829,0.062404,...,0.363005,-0.046105,-0.113011,0.136682,-0.060703,-0.126660,-0.119020,-0.120309,-0.085136,0.052103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,210429-1,0.826333,37.573935,0.104372,0.219628,0,43,Normal,0.150124,0.063731,...,0.362294,-0.045737,-0.113292,0.136584,-0.060718,-0.125992,-0.119388,-0.121251,-0.086100,0.051824
59,210407-1,0.825272,37.471294,0.104087,0.219450,0,43,Hypoglycemia,0.148133,0.063992,...,0.362685,-0.045838,-0.112205,0.136294,-0.060576,-0.125732,-0.119181,-0.120724,-0.085379,0.052402
60,210311-1,0.822136,37.324738,0.103680,0.219162,1,18,Normal,0.146501,0.058281,...,0.363047,-0.045369,-0.116989,0.134350,-0.061192,-0.128317,-0.117566,-0.115824,-0.082533,0.052055
61,210518-1,0.813073,36.805702,0.102238,0.218214,1,18,Hyperglycemia,0.147413,0.059866,...,0.363143,-0.045276,-0.116999,0.135208,-0.060912,-0.127969,-0.118160,-0.116986,-0.082722,0.051224


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['Status', 'SurID'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.6153846153846154
Extra Trees Accuracy: 0.5384615384615384


In [84]:
# Drop the SpecID column
X = joined_graph.drop(columns=['Status', 'SurID'])

# Target variable
y = joined_graph['Status']

# Creating the classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=1234)
et = ExtraTreesClassifier(n_estimators=100,random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.3238 +/- 0.1360
ExtraTreesClassifier Cross-Validation Accuracy: 0.4286 +/- 0.2244


In [49]:
evaluate_extra_trees(joined_graph)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3283 +/- 0.0461
